# VeriRegime - ONNX模型导出

将训练好的MLP Student导出为ONNX格式，用于zkML转换

**输入**：`results/checkpoints/best_student.pth`  
**输出**：`results/onnx/student_model.onnx`


## 1. 导入依赖


In [6]:
import sys
sys.path.append('..')

import torch
import torch.onnx
import numpy as np
import platform
import os

from train import MLPStudent

print(f"PyTorch版本: {torch.__version__}")
print(f"ONNX导出支持: {'✅' if hasattr(torch.onnx, 'export') else '❌'}")


PyTorch版本: 2.9.1
ONNX导出支持: ✅


## 2. 配置参数


In [7]:
# 模型配置（与训练时一致）
FEATURE_COLS = ['ema_5', 'ema_10', 'ema_20', 'rsi', 'macd', 'volume_ma_5', 'volume_ma_10']
SEQ_LENGTH = 240
INPUT_DIM = SEQ_LENGTH * len(FEATURE_COLS)  # 1680

# 路径配置
CHECKPOINT_PATH = '../results/checkpoints/best_student.pth'
ONNX_DIR = '../results/onnx'
ONNX_PATH = os.path.join(ONNX_DIR, 'student_model.onnx')

# 创建输出目录
os.makedirs(ONNX_DIR, exist_ok=True)

print(f"输入维度: {INPUT_DIM}")
print(f"Checkpoint: {CHECKPOINT_PATH}")
print(f"ONNX输出: {ONNX_PATH}")


输入维度: 1680
Checkpoint: ../results/checkpoints/best_student.pth
ONNX输出: ../results/onnx/student_model.onnx


## 3. 加载训练好的模型


In [8]:
print("\n加载MLP Student...")

# 创建模型
model = MLPStudent(
    input_dim=INPUT_DIM,
    hidden_dims=[256, 128, 64],
    dropout_rate=0.3
)

# 加载权重
checkpoint = torch.load(CHECKPOINT_PATH, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # 设为评估模式

# 统计参数
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ 模型加载成功")
print(f"  参数量: {total_params:,}")
print(f"  最佳F1: {checkpoint['best_f1']:.4f}")
print(f"  训练轮次: {checkpoint['epoch']}")



加载MLP Student...
✅ 模型加载成功
  参数量: 471,618
  最佳F1: 0.7256
  训练轮次: 25


## 4. 准备示例输入


In [9]:
print("\n准备示例输入...")

# 创建示例输入（batch_size=1，seq_length=240，features=7）
dummy_input = torch.randn(1, SEQ_LENGTH, len(FEATURE_COLS))

print(f"输入形状: {dummy_input.shape}")
print(f"输入范围: [{dummy_input.min():.2f}, {dummy_input.max():.2f}]")

# 测试模型推理
with torch.no_grad():
    output = model(dummy_input)
    logits = output['logits']
    pred = torch.argmax(logits, dim=1)

print(f"\n输出logits: {logits[0].numpy()}")
print(f"预测类别: {pred.item()} ({'HIGH' if pred.item() == 1 else 'LOW'} Volatility)")
print(f"\n✅ 模型推理正常")



准备示例输入...
输入形状: torch.Size([1, 240, 7])
输入范围: [-3.44, 3.01]

输出logits: [-1.2432619 -0.1276757]
预测类别: 1 (HIGH Volatility)

✅ 模型推理正常


## 5. 导出ONNX模型


In [ ]:
print("\n开始导出ONNX...")

# ONNX导出配置（使用PyTorch 2.9最佳实践）
input_names = ['input']
output_names = ['logits', 'features']

# 使用dynamic_shapes替代dynamic_axes（避免警告）
from torch.export import Dim
batch_dim = Dim("batch", min=1, max=1024)

dynamic_shapes = {
    "input": {0: batch_dim}
}

# 导出ONNX（使用Opset 18，EZKL完全兼容）
torch.onnx.export(
    model,
    dummy_input,
    ONNX_PATH,
    export_params=True,
    opset_version=18,  # 使用最新版本（EZKL支持12-18）
    do_constant_folding=True,
    input_names=input_names,
    output_names=output_names,
    dynamic_shapes=dynamic_shapes,  # 新API，避免警告
    dynamo=True,  # 显式启用新导出器
    verbose=False
)

print(f"✅ ONNX导出成功: {ONNX_PATH}")
print(f"  使用Opset 18（最新，无警告）")
print(f"  动态batch_size支持: 1-1024")

# 检查文件大小
file_size = os.path.getsize(ONNX_PATH) / 1024  # KB
print(f"  文件大小: {file_size:.2f} KB")


/var/folders/20/z7rcn5zx7hbcd8xbwb89m51h0000gn/T/ipykernel_49863/3896541333.py:15: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1202 10:41:56.012000 49863 site-packages/torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 14 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features



开始导出ONNX...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 14).
Failed to convert the model to the target version 14 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "/Users/lambertlin/anaconda3/envs/ml/lib/python3.12/site-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lambertlin/anaconda3/envs/ml/lib/python3.12/site-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/Users/lambertlin/anaconda3/envs/ml/lib/python3.12/site-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Applied 1 of general pattern rewrite rules.
✅ ONNX导出成功: ../results/onnx/student_model.onnx
  文件大小: 13.32 KB


## 6. 验证ONNX模型


In [14]:
print("\n验证ONNX模型...")

import onnx
import onnxruntime as ort

# 加载ONNX模型
onnx_model = onnx.load(ONNX_PATH)

# 检查模型有效性
try:
    onnx.checker.check_model(onnx_model)
    print("✅ ONNX模型格式有效")
except Exception as e:
    print(f"❌ ONNX模型检查失败: {e}")
    raise

# 打印模型信息
print(f"\n模型信息:")
print(f"  IR版本: {onnx_model.ir_version}")
print(f"  Opset版本: {onnx_model.opset_import[0].version}")
print(f"  生产者: {onnx_model.producer_name}")

print(f"\n输入:")
for input_tensor in onnx_model.graph.input:
    shape = [d.dim_value if d.dim_value > 0 else 'dynamic' for d in input_tensor.type.tensor_type.shape.dim]
    print(f"  {input_tensor.name}: {shape}")

print(f"\n输出:")
for output_tensor in onnx_model.graph.output:
    shape = [d.dim_value if d.dim_value > 0 else 'dynamic' for d in output_tensor.type.tensor_type.shape.dim]
    print(f"  {output_tensor.name}: {shape}")



验证ONNX模型...
✅ ONNX模型格式有效

模型信息:
  IR版本: 10
  Opset版本: 18
  生产者: pytorch

输入:
  input: ['dynamic', 240, 7]

输出:
  logits: ['dynamic', 2]


## 7. ONNX Runtime推理测试


In [15]:
print("\nONNX Runtime推理测试...")

# 创建ONNX Runtime会话
ort_session = ort.InferenceSession(ONNX_PATH)

# 准备输入
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}

# 运行推理
ort_outputs = ort_session.run(None, ort_inputs)
ort_logits = ort_outputs[0]

print(f"ONNX输出logits: {ort_logits[0]}")
print(f"ONNX预测类别: {np.argmax(ort_logits[0])}")

# 对比PyTorch和ONNX输出
with torch.no_grad():
    pytorch_output = model(dummy_input)['logits'].numpy()

diff = np.abs(pytorch_output - ort_logits).max()
print(f"\nPyTorch vs ONNX最大差异: {diff:.6f}")

if diff < 1e-5:
    print("✅ ONNX模型与PyTorch模型一致（差异 < 1e-5）")
elif diff < 1e-3:
    print("⚠️ ONNX模型与PyTorch有轻微差异（1e-5 < 差异 < 1e-3）")
else:
    print(f"❌ ONNX模型与PyTorch差异较大（差异 = {diff:.6f}）")



ONNX Runtime推理测试...
ONNX输出logits: [-1.243262   -0.12767568]
ONNX预测类别: 1

PyTorch vs ONNX最大差异: 0.000000
✅ ONNX模型与PyTorch模型一致（差异 < 1e-5）


## 8. 保存模型元数据


In [ ]:
import json
from datetime import datetime

metadata = {
    'export_time': datetime.now().isoformat(),
    'model': {
        'type': 'MLPStudent',
        'input_dim': INPUT_DIM,
        'hidden_dims': [256, 128, 64],
        'output_dim': 2,
        'total_params': total_params,
        'dropout_rate': 0.3
    },
    'training': {
        'best_f1': float(checkpoint['best_f1']),
        'epoch': checkpoint['epoch']
    },
    'onnx': {
        'path': ONNX_PATH,
        'opset_version': 18,  # 实际使用的版本
        'file_size_kb': file_size,
        'input_shape': [1, SEQ_LENGTH, len(FEATURE_COLS)],
        'output_shape': [1, 2],
        'dynamic_batch': True  # 支持动态batch_size
    },
    'features': FEATURE_COLS,
    'seq_length': SEQ_LENGTH
}

metadata_path = os.path.join(ONNX_DIR, 'model_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n✅ 元数据已保存: {metadata_path}")
print(f"\n{'='*60}")
print("🎉 ONNX导出完成！")
print(f"{'='*60}")
print(f"\n下一步: 使用EZKL进行zkML转换")
print(f"  1. 安装EZKL: scripts/setup_ezkl.sh")
print(f"  2. 运行转换: notebooks/zkml_pipeline.ipynb")



✅ 元数据已保存: ../results/onnx/model_metadata.json

🎉 ONNX导出完成！

下一步: 使用EZKL进行zkML转换
  1. 安装EZKL: scripts/setup_ezkl.sh
  2. 运行转换: notebooks/zkml_pipeline.ipynb
